# Sample VQE

This notebook has a sample of how to use VQE in order to solve for the conformer shown in the image below. You can use this algorithm to solve for all other confomers with angles ranging from $0-2\pi$ <img src="conf1.jpg">

## Required Imports

In [1]:
from qiskit_nature.units import DistanceUnit                                  #Angstrom unit
from qiskit_nature.second_q.drivers import PySCFDriver                        #PySCF Driver
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer        #ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import JordanWignerMapper                 #Mappers


## Loading the drivers

In [2]:
%%time
ethylene="C 0.00000000 0.00000000 0.76532900; H -0.51007400 0.88446900 1.16448900; H 1.02100900   -0.00049700    1.16448900; H -0.51093500   -0.88397100    1.16448900;  C 0.00000000 0.00000000 -0.76532900; H 0.51093500 -0.88397100 -1.164489; H -1.02100900 -0.00049700 -1.16448900; H 0.51007400 0.88446900 -1.16448900"
driver = PySCFDriver(
    
    
    atom=ethylene,
       
    
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)

es_problem = driver.run()

CPU times: user 553 ms, sys: 31.9 ms, total: 585 ms
Wall time: 3.01 s


In [3]:
%%time
problem=driver.run()
print(problem)

CPU times: user 409 ms, sys: 3.7 ms, total: 413 ms
Wall time: 411 ms


In [4]:
print(problem.num_particles)
print(problem.num_spatial_orbitals)

(9, 9)
16


## Active Space Transformer

In [5]:
as_transformer = ActiveSpaceTransformer(2, 2, active_orbitals=[8, 9]) 


In [6]:
%%time
problem2 = as_transformer.transform(problem)

CPU times: user 22.1 ms, sys: 252 µs, total: 22.4 ms
Wall time: 54.2 ms


In [7]:
print(problem2.num_particles)
print(problem2.num_spatial_orbitals)

(1, 1)
2


In [8]:
fermionic_op = problem2.hamiltonian.second_q_op()

In [9]:
%%time
qubit_jw_op = JordanWignerMapper().map(fermionic_op)

qubit_jw_op.num_qubits

CPU times: user 50.2 ms, sys: 0 ns, total: 50.2 ms
Wall time: 87.5 ms


4

In [10]:
fermionic_op.to_matrix().toarray().shape

(16, 16)

In [11]:
from qiskit_nature.second_q.mappers import QubitConverter


converter = QubitConverter(JordanWignerMapper(), z2symmetry_reduction="auto")

qubit_jw_op = converter.convert(fermionic_op, sector_locator=problem2.symmetry_sector_locator)
print(qubit_jw_op.num_qubits)

2


## VQE

In [12]:
from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.algorithms import VQEUCCFactory
from qiskit_nature.second_q.circuit.library import UCCSD

vqe_solver = VQEUCCFactory(Estimator(), UCCSD(qubit_converter=converter), SLSQP())

In [13]:
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.circuit.library import TwoLocal

tl_circuit = TwoLocal(
    rotation_blocks=["h", "rx"],
    entanglement_blocks="cz",
    entanglement="full",
    reps=2,
    parameter_prefix="y",
)

In [14]:
%%time
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(converter, vqe_solver)

CPU times: user 17 µs, sys: 3 µs, total: 20 µs
Wall time: 24.1 µs


In [15]:
%%time
res = calc.solve(problem2)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -120.411405723879
  - computed part:      -1.298683705436
  - ActiveSpaceTransformer extracted energy part: -119.112722018443
~ Nuclear repulsion energy (Hartree): 42.105607210653
> Total ground state energy (Hartree): -78.305798513227
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.00000378  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.00000438  0.0]
    - computed part:      [0.0  -0.00156427  0.0]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.00156865  0.0]
  > Dipole moment (a.u.): [0.0  -0.0000006  0.0]  Total: 0.0000006
                 (debye): [0.0  -0.00000153  0.0]  Total: 0.00000153
 
CPU times: user 308 ms, sys: 15.7 ms, total: 324 ms
Wall time: 414 ms
